In [29]:
import nltk
import threading
import queue
import feedparser
import uuid

In [30]:
threads = []
queues =[queue.Queue() ,queue.Queue()]

In [31]:
def extractWords():
    url = 'http://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid':uuid.uuid4(), 'input':words}
        queues[0].put(data,True)
        print(">> {} : {}".format(data['uuid'],text))

In [32]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid':data['uuid'], 'input':postags},True)

In [33]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary = False)
            print("<< {} : ".format(data['uuid']), end='')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end =', ')
                except:
                    pass
            print()

In [34]:
def runProgram():
    e = threading.Thread(target = extractWords())
    e.start()
    threads.append(e)
    
    p = threading.Thread(target = extractPOS())
    p.start()
    threas.append(p)
    
    n = threading.Thread(target = extractNE())
    n.start()
    threads.append(n)
    
    queues[0].join()
    queues[1].join()
    
    for t in threads:
        t.join()

In [35]:
if __name__ == '__main__':
    runProgram()

>> fc3caf4c-69c2-4ba0-bf57-7b67cdece989 : Mahesh Manjrekar: I feel Salman Khan is lonely
>> 16b089fe-fbbf-475e-a951-a7485a78ec0e : Karan recalls his first meeting with SRK
>> e473f1d8-12d7-4cc9-8cf8-2c519253b437 : Dia 'grateful' for first Diwali with son
>> 0aeebf95-a9a2-41f5-a293-8349bc046ab5 : 5 firecracker statements on B'wood by Nawaz
>> 8754d16b-6c0c-4571-8133-8c9cc4f5bca8 : Pic: Fan pretends to be SRK, greets visitors
<< fc3caf4c-69c2-4ba0-bf57-7b67cdece989 : (PERSON Mahesh/JJ Manjrekar/NNP), (PERSON Salman/NNP Khan/NNP), 
<< 16b089fe-fbbf-475e-a951-a7485a78ec0e : (GPE Karan/NNP), (ORGANIZATION SRK/NNP), 
<< e473f1d8-12d7-4cc9-8cf8-2c519253b437 : (GPE Dia/NNP), 
<< 0aeebf95-a9a2-41f5-a293-8349bc046ab5 : (PERSON Nawaz/NNP), 
<< 8754d16b-6c0c-4571-8133-8c9cc4f5bca8 : (ORGANIZATION SRK/NNP), 
